In [1]:
from transformers import RobertaTokenizerFast, RobertaForTokenClassification, Trainer, TrainingArguments
import torch
from torch.utils.data._utils.collate import default_convert

In [2]:
model = RobertaForTokenClassification.from_pretrained('/home/allekim/stonybook-dev/hathi_similarity/new_ocr_detection/tok_model')

In [3]:
device = 'cuda:2'
model.to(device)

RobertaForTokenClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
            

In [4]:
model.eval()

RobertaForTokenClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
            

In [5]:
dataset = torch.load('/home/allekim/stonybook-data/guten_hathi_alignment/2735/uva.x000875532_hathi_input.dt')

In [6]:
results = torch.load('/home/allekim/stonybook-data/guten_hathi_alignment/2735/uva.x000875532_hathi_results.dt')

In [7]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, collate_fn=default_convert)

In [8]:
results = []
for i, batch in enumerate(dataloader):
    input_ids = torch.stack([torch.tensor(x['input_ids']) for x in batch]).to(device)
    attention_mask = torch.stack([torch.tensor(x['attention_mask']) for x in batch]).to(device)
    x = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = x.logits.detach().cpu().numpy()
    results.append(logits)
    if i > 2:
        break

In [17]:
results[0].shape

(4, 512, 2)

In [19]:
import numpy as np

In [21]:
y = np.concatenate(results)

In [22]:
y.shape

(16, 512, 2)